# Lab 3.6.5: CrewAI - Production Role-Based Agent Teams

**Module:** 3.6 - AI Agents & Agentic Systems  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the CrewAI framework for role-based agents
- [ ] Create agents with roles, goals, and backstories
- [ ] Define tasks with dependencies and context
- [ ] Build and run a production-ready crew
- [ ] Handle errors and manage crew execution

---

## 📚 Prerequisites

- Completed: Lab 3.6.4 (Multi-Agent Systems)
- Knowledge of: Agent patterns, Python dataclasses
- Running: Ollama with `qwen3:8b` or larger model

---

## 🌍 Real-World Context

**CrewAI** is a production framework for building role-based agent teams:
- **Agents** have roles, goals, and backstories (like employees)
- **Tasks** define specific work with expected outputs
- **Crews** orchestrate agents working together
- **Tools** give agents real-world capabilities

**Why CrewAI?**
- Production-ready with error handling
- Hierarchical and sequential workflows
- Built-in memory and context management
- Easy integration with LangChain tools

---

## 🧒 ELI5: What is CrewAI?

> **Imagine you're running a startup company...** 🏢
>
> You don't just have "workers" - you have:
> - **Roles**: Marketing Manager, Developer, Designer
> - **Goals**: Each person has specific objectives
> - **Backstory**: Experience and skills they bring
>
> When a project comes in:
> 1. You break it into **Tasks**
> 2. Assign tasks to the right **Agents**
> 3. The **Crew** works together to complete them
>
> **CrewAI automates this for AI agents!**
> - Define your team (agents with roles)
> - Define the project (tasks with outputs)
> - Run the crew and get results!

```
┌─────────────────────────────────────────────────────────────────┐
│                      CrewAI Architecture                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌───────────────────┐                                         │
│   │       CREW        │ ← Orchestrates everything               │
│   └─────────┬─────────┘                                         │
│             │                                                   │
│     ┌───────┴───────┐                                           │
│     │               │                                           │
│   ┌─┴─┐           ┌─┴─┐                                         │
│   │AGT│           │AGT│           ┌───┐                         │
│   │ 1 │           │ 2 │    ...    │AGT│                         │
│   └─┬─┘           └─┬─┘           │ N │                         │
│     │               │             └───┘                         │
│   ┌─┴───────────────┴───────────────────┐                       │
│   │              TASKS                   │                       │
│   │  ┌────┐  ┌────┐  ┌────┐  ┌────┐     │                       │
│   │  │T1  │→ │T2  │→ │T3  │→ │T4  │     │                       │
│   │  └────┘  └────┘  └────┘  └────┘     │                       │
│   └─────────────────────────────────────┘                       │
│                                                                 │
│   Each Agent has: Role, Goal, Backstory, Tools                  │
│   Each Task has: Description, Agent, Expected Output            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

**Note:** CrewAI requires installation. If not installed, we'll create a compatible simulation.

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
from datetime import datetime
import json

# Add scripts to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

print("✅ Standard imports successful!")

In [ ]:
# Try to import CrewAI, fall back to simulation if not available
try:
    from crewai import Agent, Task, Crew, Process
    from crewai.tools import tool as crewai_tool
    CREWAI_AVAILABLE = True
    print("✅ CrewAI imported successfully!")
except ImportError:
    CREWAI_AVAILABLE = False
    print("⚠️  CrewAI not installed. Using simulation mode.")
    print("   To install: pip install crewai crewai-tools")

# LangChain for LLM
from langchain_community.llms import Ollama

# Initialize LLM
llm = Ollama(
    model="qwen3:8b",
    temperature=0.7,
    base_url="http://localhost:11434"
)

print("✅ LLM initialized!")

---

## Part 2: CrewAI Simulation (for learning without installation)

Let's create a CrewAI-compatible simulation that teaches the same concepts.

In [ ]:
# CrewAI-compatible simulation classes

@dataclass
class SimAgent:
    """
    Simulated CrewAI Agent.
    
    In real CrewAI:
        from crewai import Agent
        agent = Agent(role=..., goal=..., backstory=...)
    """
    role: str
    goal: str
    backstory: str
    verbose: bool = True
    allow_delegation: bool = True
    tools: List[Callable] = field(default_factory=list)
    llm: Any = None
    
    def __post_init__(self):
        if self.llm is None:
            self.llm = llm  # Use global LLM
    
    def execute(self, task_description: str, context: str = "") -> str:
        """Execute a task."""
        prompt = f"""You are a {self.role}.

Your goal: {self.goal}

Your background: {self.backstory}

Context from previous tasks:
{context if context else 'None'}

Your current task:
{task_description}

Complete this task thoroughly:"""
        
        if self.verbose:
            print(f"\n🤖 [{self.role}] Working on task...")
        
        result = self.llm.invoke(prompt)
        return result.strip()


@dataclass
class SimTask:
    """
    Simulated CrewAI Task.
    
    In real CrewAI:
        from crewai import Task
        task = Task(description=..., agent=..., expected_output=...)
    """
    description: str
    agent: SimAgent
    expected_output: str
    context: List['SimTask'] = field(default_factory=list)
    output: str = ""


@dataclass
class SimCrew:
    """
    Simulated CrewAI Crew.
    
    In real CrewAI:
        from crewai import Crew, Process
        crew = Crew(agents=[...], tasks=[...], process=Process.sequential)
    """
    agents: List[SimAgent]
    tasks: List[SimTask]
    verbose: bool = True
    process: str = "sequential"  # sequential or hierarchical
    
    def kickoff(self, inputs: Dict[str, str] = None) -> str:
        """
        Execute all tasks in sequence.
        
        Args:
            inputs: Dictionary of input variables
        
        Returns:
            Final task output
        """
        inputs = inputs or {}
        
        if self.verbose:
            print("\n" + "="*60)
            print("🚀 CREW STARTING")
            print("="*60)
            print(f"   Agents: {[a.role for a in self.agents]}")
            print(f"   Tasks: {len(self.tasks)}")
            print(f"   Process: {self.process}")
        
        # Execute tasks in order
        for i, task in enumerate(self.tasks):
            if self.verbose:
                print(f"\n{'─'*40}")
                print(f"📋 TASK {i+1}: {task.description[:50]}...")
                print(f"   Assigned to: {task.agent.role}")
            
            # Gather context from previous tasks
            context_parts = []
            for ctx_task in task.context:
                if ctx_task.output:
                    context_parts.append(f"[{ctx_task.agent.role}]: {ctx_task.output}")
            context = "\n\n".join(context_parts)
            
            # Substitute inputs in description
            description = task.description
            for key, value in inputs.items():
                description = description.replace(f"{{{key}}}", value)
            
            # Execute task
            task.output = task.agent.execute(description, context)
            
            if self.verbose:
                print(f"   ✅ Complete ({len(task.output)} chars)")
        
        if self.verbose:
            print("\n" + "="*60)
            print("🎉 CREW COMPLETE")
            print("="*60)
        
        # Return last task output
        return self.tasks[-1].output if self.tasks else ""


# Use real CrewAI if available, otherwise use simulation
if CREWAI_AVAILABLE:
    CrewAgent = Agent
    CrewTask = Task
    CrewCrew = Crew
    print("\n✅ Using real CrewAI!")
else:
    CrewAgent = SimAgent
    CrewTask = SimTask
    CrewCrew = SimCrew
    print("\n✅ Using CrewAI simulation!")

---

## Part 3: Creating the Research Crew

Let's build a research and writing crew with three specialized agents.

In [ ]:
# Create the Research Agent
researcher = CrewAgent(
    role="Senior Research Analyst",
    goal="Uncover comprehensive, accurate information on the given topic",
    backstory="""You are an experienced research analyst with a PhD in Data Science.
You have worked at top research institutions and are known for your ability
to find hidden insights in complex topics. You always verify facts from
multiple sources and present information in a structured way.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

print("✅ Researcher agent created!")
print(f"   Role: {researcher.role}")
print(f"   Goal: {researcher.goal[:60]}...")

In [ ]:
# Create the Writer Agent
writer = CrewAgent(
    role="Senior Content Writer",
    goal="Create engaging, well-structured content that explains complex topics clearly",
    backstory="""You are a seasoned content writer with 10+ years of experience
writing for tech publications. You have a talent for making complex topics
accessible to general audiences while maintaining technical accuracy.
Your writing is always engaging, well-organized, and informative.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

print("✅ Writer agent created!")
print(f"   Role: {writer.role}")

In [ ]:
# Create the Editor Agent
editor = CrewAgent(
    role="Chief Editor",
    goal="Ensure content is polished, accurate, and publication-ready",
    backstory="""You are a chief editor at a major tech publication.
With your keen eye for detail, you catch errors others miss.
You improve content while preserving the author's voice.
You also ensure all claims are supported by the research.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

print("✅ Editor agent created!")
print(f"   Role: {editor.role}")

---

## Part 4: Defining Tasks

Tasks define what each agent should do, with clear expected outputs.

In [ ]:
# Task 1: Research
research_task = CrewTask(
    description="""Research the topic: {topic}
    
Your research should include:
1. Overview of the topic
2. Key facts and statistics
3. Current trends and developments
4. Expert opinions and perspectives
5. Potential challenges or controversies

Organize your findings with clear headings and bullet points.""",
    expected_output="A comprehensive research report with organized findings, facts, and insights.",
    agent=researcher
)

print("✅ Research task created!")

In [ ]:
# Task 2: Write article
writing_task = CrewTask(
    description="""Write a comprehensive article about: {topic}
    
Use the research provided to create an engaging article that:
1. Has a compelling introduction
2. Explains the topic clearly for general readers
3. Uses examples and analogies
4. Includes practical implications
5. Has a strong conclusion

Target length: 500-800 words.""",
    expected_output="An engaging, well-structured article suitable for publication.",
    agent=writer,
    context=[research_task]  # Uses research output as context
)

print("✅ Writing task created!")
print("   (depends on research task)")

In [ ]:
# Task 3: Edit and polish
editing_task = CrewTask(
    description="""Review and edit the article about: {topic}
    
Your editing should:
1. Fix any grammar or spelling errors
2. Improve sentence structure and flow
3. Verify claims against the research
4. Enhance readability and engagement
5. Ensure consistent tone and style

Return the final, polished version ready for publication.""",
    expected_output="A polished, publication-ready article.",
    agent=editor,
    context=[research_task, writing_task]  # Has access to both
)

print("✅ Editing task created!")
print("   (depends on research and writing tasks)")

---

## Part 5: Building and Running the Crew

In [ ]:
# Create the crew
research_crew = CrewCrew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    verbose=True,
    process="sequential"  # Tasks run in order
)

print("✅ Research crew assembled!")
print(f"   Team: {[a.role for a in research_crew.agents]}")
print(f"   Tasks: {len(research_crew.tasks)}")

In [ ]:
# Run the crew!
import time

print("\n🚀 Starting the crew...")
start_time = time.time()

# Kick off with input variables
result = research_crew.kickoff(
    inputs={
        "topic": "The impact of local AI computing with devices like DGX Spark on enterprise AI adoption"
    }
)

duration = time.time() - start_time

print(f"\n⏱️  Total time: {duration:.1f} seconds")

In [ ]:
# View the final output
print("\n" + "="*60)
print("📰 FINAL ARTICLE")
print("="*60)
print(result)

---

## Part 6: Advanced - Crew with Custom Tools

Let's add custom tools that agents can use.

In [ ]:
# Define custom tools
from langchain.tools import tool

@tool
def search_web(query: str) -> str:
    """Search the web for information. Use for finding current facts."""
    # Simulated search
    results = {
        "dgx spark": "DGX Spark: 128GB unified memory, Blackwell GPU, 1 PFLOP FP4",
        "ai adoption": "83% of enterprises plan to increase AI investment by 2025",
        "local ai": "Local AI reduces data transfer costs by 60% on average",
    }
    query_lower = query.lower()
    for key, value in results.items():
        if key in query_lower:
            return f"Search result: {value}"
    return f"No specific results for: {query}"


@tool
def calculate_stat(expression: str) -> str:
    """Calculate a mathematical expression. Use for statistics and numbers."""
    try:
        # Safe evaluation
        import ast
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Calculation result: {result}"
    except:
        return f"Could not calculate: {expression}"


print("✅ Custom tools defined!")
print(f"   - {search_web.name}: {search_web.description}")
print(f"   - {calculate_stat.name}: {calculate_stat.description}")

In [ ]:
# Create an agent with tools
research_agent_with_tools = CrewAgent(
    role="Research Analyst with Tools",
    goal="Find accurate, up-to-date information using available tools",
    backstory="""You are a research analyst who knows how to use specialized
tools to find and verify information. You always use your tools to
get accurate data rather than relying on assumptions.""",
    tools=[search_web, calculate_stat],
    verbose=True,
    llm=llm
)

print("✅ Agent with tools created!")
print(f"   Tools: {[t.name for t in research_agent_with_tools.tools]}")

---

## Part 7: Production Patterns

### Pattern 1: Hierarchical Process

In [ ]:
# In hierarchical process, a manager agent delegates to workers

manager = CrewAgent(
    role="Project Manager",
    goal="Coordinate the team to deliver high-quality results",
    backstory="""You are an experienced project manager who knows how
to delegate tasks effectively and ensure quality deliverables.
You coordinate between team members and resolve conflicts.""",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

print("✅ Manager agent created for hierarchical crew!")
print("""
📊 Hierarchical Process:
   - Manager receives the overall goal
   - Manager delegates specific tasks to specialists
   - Manager reviews and combines outputs
   - Useful for complex, multi-part projects
""")

### Pattern 2: Callback Functions

In [ ]:
# Callbacks let you track crew progress

def on_task_start(task):
    """Called when a task starts."""
    print(f"📋 Starting: {task.description[:30]}...")

def on_task_complete(task, output):
    """Called when a task completes."""
    print(f"✅ Completed: {task.description[:30]}...")
    print(f"   Output length: {len(output)} chars")

def on_crew_complete(crew, outputs):
    """Called when the entire crew finishes."""
    print(f"🎉 Crew finished with {len(outputs)} task outputs!")

print("""
📊 Callback Pattern:
   - on_task_start: Log, notify, start timer
   - on_task_complete: Save output, update status
   - on_crew_complete: Send notifications, cleanup
""")

### Pattern 3: Error Handling

In [ ]:
def run_crew_with_retry(crew, inputs, max_retries=3):
    """
    Run a crew with automatic retry on failure.
    
    Args:
        crew: The crew to run
        inputs: Input dictionary
        max_retries: Maximum retry attempts
    
    Returns:
        Crew output or None if all retries fail
    """
    for attempt in range(max_retries):
        try:
            print(f"\n🔄 Attempt {attempt + 1}/{max_retries}")
            result = crew.kickoff(inputs=inputs)
            print("✅ Success!")
            return result
        except Exception as e:
            print(f"❌ Error: {e}")
            if attempt < max_retries - 1:
                print("   Retrying...")
                import time
                time.sleep(2)  # Wait before retry
            else:
                print("   Max retries reached.")
    
    return None

print("✅ Retry pattern defined!")

---

## ⚠️ Common Mistakes

### Mistake 1: Vague Agent Goals

In [ ]:
print("""
❌ WRONG: Vague goal

agent = Agent(
    role="Helper",
    goal="Help with stuff",  # Too vague!
    backstory="I help."
)

✅ RIGHT: Specific, measurable goal

agent = Agent(
    role="Technical Documentation Writer",
    goal="Create clear, accurate documentation that enables developers to implement features correctly on first attempt",
    backstory="10 years experience writing API docs for major tech companies..."
)
""")

### Mistake 2: Missing Task Context

In [ ]:
print("""
❌ WRONG: Writer doesn't have research context

write_task = Task(
    description="Write about AI",
    agent=writer
    # No context! Writer has to make things up.
)

✅ RIGHT: Pass research as context

write_task = Task(
    description="Write about AI using the provided research",
    agent=writer,
    context=[research_task]  # Writer sees research output!
)
""")

---

## 🎉 Checkpoint

You've learned:
- ✅ CrewAI concepts: Agents, Tasks, Crews
- ✅ Defining agents with roles, goals, and backstories
- ✅ Creating tasks with dependencies
- ✅ Running sequential and hierarchical crews
- ✅ Adding custom tools to agents
- ✅ Production patterns for error handling

---

## ✋ Try It Yourself

1. Create a **Marketing Crew** with: Strategist, Copywriter, Designer (text descriptions)
2. Add a **Quality Assurance** agent that validates the final output
3. Implement a **parallel review** where two editors review simultaneously

---

## 🚀 Challenge (Optional)

Build a **Customer Support Crew**:
- Triage Agent: Categorizes incoming requests
- Technical Agent: Handles technical issues
- Billing Agent: Handles billing questions
- Escalation Agent: Handles complex cases

Use hierarchical process with a Support Manager.

---

## 📖 Further Reading

- [CrewAI Documentation](https://docs.crewai.com/)
- [CrewAI GitHub](https://github.com/joaomdmoura/crewAI)
- [CrewAI Examples](https://github.com/joaomdmoura/crewAI-examples)

---

## 🧹 Cleanup

In [ ]:
import gc
gc.collect()

try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
except:
    pass

print("✅ Cleanup complete!")

---

## 🎓 Summary

| Component | Purpose | Key Attributes |
|-----------|---------|----------------|
| **Agent** | Specialized worker | role, goal, backstory, tools |
| **Task** | Unit of work | description, agent, expected_output, context |
| **Crew** | Team orchestrator | agents, tasks, process (sequential/hierarchical) |

**CrewAI Pattern:**
```python
# 1. Define agents with clear roles
researcher = Agent(role="...", goal="...", backstory="...")

# 2. Define tasks with dependencies
task = Task(description="...", agent=researcher, context=[...])

# 3. Create and run crew
crew = Crew(agents=[...], tasks=[...])
result = crew.kickoff(inputs={"topic": "..."})
```

**Next up:** Lab 3.6.6 - Agent Benchmarking and Evaluation